# Sheet 11: 

In moodle you can find the file arXiv train.csv and arXiv test.csv. It contains the titles,
abstracts and categories of 107,055 papers uploaded to the preprint-platform arXiv. You are
tasked to create a pipeline, which predicts the category (given in the terms column) of a paper.
The train corpus contains 100 labeled examples for the 4 classes.

# Task 1

# Argue the usefulness of the following models for the task at hand in 2-3 sentences:
• Dictionary-based analysis
• Latent Dirichlet Allocation
• Word2Vec
• Doc2Vec
• BERT
# Are there other analyses that might help to solve this task? Which analyses do you deem tobe the best for this task?

# Task 1 
Argue the usefulness of the following models for the task at hand in 2-3 sentences:

# Task 2

Preprocess the data so that it is fit for the pipeline you intend to solve the task with. Shortly
explain every step of preprocessing and why it is useful for this analysis.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load the data
train_data = pd.read_csv("/Users/oayanwale/Downloads/NLP_Exercise_23/arxiv_train.csv")
test_data = pd.read_csv("/Users/oayanwale/Downloads/NLP_Exercise_23/arxiv_test.csv")

# Display the first few rows of the dataset
print(train_data.head())
print(test_data.head())

# Print the columns to check their names
print("Train Data Columns:", train_data.columns)
print("Test Data Columns:", test_data.columns)

                                              titles  \
0  Nearly Minimax Optimal Reinforcement Learning ...   
1  An Intrinsically-Motivated Approach for Learni...   
2  Adversarial Active Exploration for Inverse Dyn...   
3                 Accelerated Reinforcement Learning   
4  Learning Adaptive Display Exposure for Real-Ti...   

                                           abstracts terms  
0  We study reinforcement learning (RL) with line...  math  
1  What is a good exploration strategy for an age...  stat  
2  We present an adversarial active exploration f...  stat  
3  Policy gradient methods are widely used in rei...    cs  
4  In E-commerce advertising, where product recom...  stat  
                                              titles  \
0  Change your singer: a transfer learning genera...   
1  Towards Understanding Pixel Vulnerability unde...   
2  Pedestrian Detection with Spatially Pooled Fea...   
3  Relative concentration bounds for the spectrum...   
4  Enhancing Traj

In [3]:
#option 1 the best

import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Ensure required NLTK resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize text into words
    tokens = [re.sub(r'[^a-zA-Z]', '', word) for word in tokens]  # Remove special characters and numbers
    tokens = [word for word in tokens if word and word not in stopwords.words('english')]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Apply lemmatization
    return ' '.join(tokens)

# Load the dataset
train_data = pd.read_csv("/Users/oayanwale/Downloads/NLP_Exercise_23/arxiv_train.csv")
test_data = pd.read_csv("/Users/oayanwale/Downloads/NLP_Exercise_23/arxiv_test.csv")

# Apply preprocessing to titles and abstracts
train_data['processed_titles'] = train_data['titles'].apply(preprocess_text)
train_data['processed_abstracts'] = train_data['abstracts'].apply(preprocess_text)

test_data['processed_titles'] = test_data['titles'].apply(preprocess_text)
test_data['processed_abstracts'] = test_data['abstracts'].apply(preprocess_text)

# Combine processed titles and abstracts into a single text column
train_data['text'] = train_data['processed_titles'] + " " + train_data['processed_abstracts']
test_data['text'] = test_data['processed_titles'] + " " + test_data['processed_abstracts']

# Split training data into training and validation sets (80-20 split)
train_set, val_set = train_test_split(train_data, test_size=0.2, random_state=42)

# Print dataset info
print("Training samples:", len(train_set))
print("Validation samples:", len(val_set))

# Print first few processed rows
print(train_set[['text', 'terms']].head())


[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Training samples: 80
Validation samples: 20
                                                 text terms
55  accelerating deep unsupervised domain adaptati...    cs
88  improved part segmentation performance optimis...    cs
26  attentive relational network mapping image sce...    cs
42  transfer learning aided target recognition com...    cs
69  whereandwhen look deep siamese attention netwo...    cs


In [11]:
# option 2 

# Preprocessing to prepare for furtherb analysis(raw datan into clean data)
def preprocess_text(text):
    # Lowercasing for uniformity and consistency
    text = text.lower()
    # Remove special characters and numbers to reduce noise
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    # Tokenization to split into individual word or tokens
    tokens = word_tokenize(text) 
    # Stopword removal to remove irrelavant words and reduce noise
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization reduce word to their roof form
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Stemming to reduce word to their roof form
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing to titles and abstracts separately using correct column names
train_data['processed_titles'] = train_data['titles'].apply(preprocess_text)
train_data['processed_abstracts'] = train_data['abstracts'].apply(preprocess_text)

test_data['processed_titles'] = test_data['titles'].apply(preprocess_text)
test_data['processed_abstracts'] = test_data['abstracts'].apply(preprocess_text)

# Combine processed titles and abstracts for modeling purposes:
train_data['text'] = train_data['processed_titles'] + " " + train_data['processed_abstracts']
test_data['text'] = test_data['processed_titles'] + " " + test_data['processed_abstracts']

# Split into training and validation sets (80-20 split) from training data only
train_set, val_set = train_test_split(train_data, test_size=0.2, random_state=42)

print("Training samples:", len(train_set))
print("Validation samples:", len(val_set))

# Optional: Print first few rows of processed data for verification 
print(train_set[['text', 'terms']].head())

Training samples: 80
Validation samples: 20
                                                 text terms
55  acceler deep unsupervis domain adapt transfer ...    cs
88  improv part segment perform optimis realism sy...    cs
26  attent relat network map imag scene graph scen...    cs
42  transfer learn aid target recognit compar deep...    cs
69  whereandwhen look deep siames attent network v...    cs


# Shortly explain every step of preprocessing and why it is useful for this analysis.

Lowercasing: This step converts all characters in the text to lowercase. It ensures uniformity and consistency, preventing issues where the same word in different cases (e.g., "Deep" vs. "deep") is treated as distinct tokens during analysis.

Removing Special Characters and Numbers: By removing special characters and numbers, we reduce noise in the data that may not contribute meaningfully to understanding the content. This helps focus on relevant words that carry semantic weight


Tokenization: Tokenization splits the cleaned text into individual words or tokens. This is essential for further processing steps like stopword removal, stemming, or lemmatization, as it allows you to operate on each word individually.

Stopword Removal: Stopwords are common words (e.g., "and," "the," "is") that typically do not add significant meaning to texts and can be safely removed to reduce noise. This helps improve model performance by focusing on more meaningful terms related to classification tasks.

Lemmatization: Lemmatization reduces words to their base or root form (e.g., "running" becomes "run"). Unlike stemming, which may produce non-words (e.g., "better" → "better"), lemmatization maintains valid dictionary forms. This step enhances semantic understanding by grouping different forms of a word together.

Stemming: Stemming further reduces words to their root form but does so more aggressively than lemmatization (e.g., “running” → “run,” “better” → “better”). While stemming can sometimes lead to less accurate representations, it’s useful when you want a more generalized representation of terms across documents.


Combining Processed Texts: After processing titles and abstracts separately, they are combined into one string to creates a single textual representation that captures all relevant information about each paper, making it easier for models to learn from both components simultaneously during training.


Splitting Data into Training and Validation Sets: Splitting the dataset allows us to evaluate model performance on unseen data after training without overfitting on the entire dataset. The validation set serves as a benchmark to assess how well your model generalizes beyond its training examples.


Summary of Importance:
✅ Lowercasing: Standardizes text and avoids case-related discrepancies.
✅ Removing special characters and numbers: Helps in reducing noise.
✅ Tokenization: Splitting text into individual words improves analysis.
✅ Stopword removal: Eliminates common but non-informative words.
✅ Lemmatization & Stemming: Helps to reduce words to their base/root form, improving model generalization.
✅ Combining titles & abstracts: Creates a richer representation of the research paper for classification.
✅ Training-validation split: Ensures model evaluation is done on unseen data.

# Task 3

Solve the task by creating a pipeline with the model you deem to be optimal for this task.

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

In [12]:
#option 1

# BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode labels
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['terms'])

# Tokenization and padding
max_length = 128
train_encodings = tokenizer(list(train_data['titles'] + ' ' + train_data['abstracts']), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(list(test_data['titles'] + ' ' + test_data['abstracts']), truncation=True, padding=True, max_length=max_length)

# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_data['label'].values
))

# BERT Model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Training
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

# Prediction
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    np.zeros(len(test_data))  # Dummy labels, as they are not used during prediction
))
predictions = model.predict(test_dataset.batch(16))
predicted_labels = np.argmax(predictions.logits, axis=1)
predicted_categories = label_encoder.inverse_transform(predicted_labels)

# Add predicted categories to test_data
test_data['predicted_category'] = predicted_categories 

# Save results to CSV
test_data.to_csv("arXiv_test_with_predictions.csv", index=False)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
7/7 [==============================] - 26s 3s/step - loss: 1.2112 - accuracy: 0.4300
Epoch 2/3
7/7 [==============================] - 18s 3s/step - loss: 0.9822 - accuracy: 0.6400
Epoch 3/3
6685/6685 [==============================] - 5769s 863ms/step


In [17]:
# option 2
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load BERT tokenizer and model 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(train_set['terms'].unique()))  # Adjust num_labels based on your classes

# Create a label mapping
label_mapping = {label: idx for idx, label in enumerate(train_set['terms'].unique())}

# Tokenize input texts for BERT model 
def tokenize_function(examples):
    return tokenizer([example['text'] for example in examples], padding='max_length', truncation=True)

# Prepare datasets for training and validation
train_encodings = tokenize_function(train_set.to_dict(orient='records'))
val_encodings = tokenize_function(val_set.to_dict(orient='records'))

class ArxivDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        # Convert string labels to numerical values using label mapping
        self.labels = [label_mapping[label] for label in labels]
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

# Create dataset instances with numerical labels
train_dataset = ArxivDataset(train_encodings, train_set['terms'].values)  # Ensure 'terms' contains correct category labels
val_dataset = ArxivDataset(val_encodings, val_set['terms'].values)

# Set up training arguments 
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model 
trainer.train()

# Evaluate the model 
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/30 [00:00<?, ?it/s]

{'train_runtime': 45.3315, 'train_samples_per_second': 5.294, 'train_steps_per_second': 0.662, 'train_loss': 1.4401972452799479, 'epoch': 3.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3773667812347412,
 'eval_runtime': 1.3815,
 'eval_samples_per_second': 14.477,
 'eval_steps_per_second': 2.172,
 'epoch': 3.0}

Training Metrics
Train Runtime: The total time taken for training (in seconds).
Train Samples per Second: The number of samples processed per second during training.
Train Steps per Second: The number of training steps completed per second.
Train Loss: The average loss value during training; lower values indicate better performance as it means the model's predictions are closer to the actual labels.
Epoch: Indicates that the training ran through all samples in your dataset for three complete passes (epochs).

Evaluation Metrics
Eval Loss: The average loss value calculated on your validation set after training; similar to train loss, lower values indicate better performance.
Eval Runtime: Total time taken for evaluation (in seconds).
Eval Samples per Second: Number of samples processed per second during evaluation.
Eval Steps per Second: Number of evaluation steps completed per second.
Epoch: Indicates that this evaluation occurred after completing three epochs.

Summary
Overall, these metrics suggest that our model has been trained successfully over three epochs with reasonable runtime and loss values for both training and evaluation phases.




In [4]:
# option 3 the best
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_labels = len(train_set['terms'].unique())  # Ensure num_labels match unique categories
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a label mapping
label_mapping = {label: idx for idx, label in enumerate(train_set['terms'].unique())}

# Tokenize function for dataset
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, return_tensors="pt")

# Tokenize training and validation data
train_encodings = tokenize_function(train_set['text'].tolist())
val_encodings = tokenize_function(val_set['text'].tolist())

# Custom PyTorch dataset class
class ArxivDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor([label_mapping[label] for label in labels], dtype=torch.long)
        
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    
    def __len__(self):
        return len(self.labels)

# Create dataset instances
train_dataset = ArxivDataset(train_encodings, train_set['terms'].values)
val_dataset = ArxivDataset(val_encodings, val_set['terms'].values)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Ensure periodic evaluation
    save_strategy="epoch",  # Save at each epoch
    load_best_model_at_end=True,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train model
trainer.train()

# Evaluate model
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.385432243347168, 'eval_runtime': 1.3122, 'eval_samples_per_second': 15.242, 'eval_steps_per_second': 2.286, 'epoch': 1.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.2857633829116821, 'eval_runtime': 0.8658, 'eval_samples_per_second': 23.1, 'eval_steps_per_second': 3.465, 'epoch': 2.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.242681622505188, 'eval_runtime': 0.8739, 'eval_samples_per_second': 22.887, 'eval_steps_per_second': 3.433, 'epoch': 3.0}
{'train_runtime': 52.7304, 'train_samples_per_second': 4.551, 'train_steps_per_second': 0.569, 'train_loss': 1.2927488962809244, 'epoch': 3.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.242681622505188,
 'eval_runtime': 0.8897,
 'eval_samples_per_second': 22.481,
 'eval_steps_per_second': 3.372,
 'epoch': 3.0}

Loss is Decreasing:
Epoch 1 → eval_loss: 1.3854
Epoch 2 → eval_loss: 1.2857
Epoch 3 → eval_loss: 1.2426
So our model is learning! 🎉

Training Speed
Our model trained 30 steps in 52.7s, which is reasonable given BERT's complexity.
Train Loss (1.2927) is close to Eval Loss (1.2426), suggesting no severe overfitting.

Next Steps for Improvement:
✅ Increase Epochs (if needed):
The loss is still decreasing, so more epochs (e.g., num_train_epochs=5) might improve results.

✅ Use Learning Rate Scheduling & Early Stopping:
A smaller learning rate (learning_rate=5e-5) with early stopping can prevent overfitting.

✅ Optimize Batch Size for Speed:
If you have enough memory, increase per_device_train_batch_size=16 to speed up training.



In [8]:
# Check Model Performance on Test Data

# Tokenize test data
test_encodings = tokenize_function(test_data['text'].tolist())  # Ensure a list of strings is passed


# Create test dataset instance
test_dataset = ArxivDataset(test_encodings, test_data['terms'].values)

# Evaluate model on test dataset
trainer.predict(test_dataset)


  0%|          | 0/13370 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 0.66824746,  0.13974105,  0.23510711, -0.70673525],
       [ 0.62987137,  0.1361922 ,  0.22343136, -0.73542416],
       [ 0.6403691 ,  0.17562026,  0.24784558, -0.6988257 ],
       ...,
       [ 0.60470325,  0.11864298,  0.22632928, -0.70553744],
       [ 0.5936044 ,  0.13653181,  0.23116635, -0.7266377 ],
       [ 0.65835637,  0.17463589,  0.22392042, -0.7050129 ]],
      dtype=float32), label_ids=array([3, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 1.1664959192276, 'test_runtime': 4776.4558, 'test_samples_per_second': 22.392, 'test_steps_per_second': 2.799})